In [ ]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

In [ ]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [ ]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [ ]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

In [ ]:
from mlflow.tracking import MlflowClient


In [ ]:
mlflow.list_experiments()

In [ ]:
mlflow.list_run_infos(experiment_id='1')

In [ ]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '62953e0cf15a48dd8434001c4b5af3b4'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [20]:
path = client.download_artifacts(run_id=RUN_ID, path='model')

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

IsADirectoryError: [Errno 21] Is a directory: '/var/folders/9j/x_25c3jx2x364k81mwrynxcm0000gn/T/tmpi8wrimlw/model'

In [21]:
path

'/var/folders/9j/x_25c3jx2x364k81mwrynxcm0000gn/T/tmpi8wrimlw/model'